In [29]:
import transformers
from datasets import load_dataset, load_metric
from datasets.dataset_dict import DatasetDict
import datasets
import random
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import argparse
import wandb
from typing import Tuple
import torch
from torch.nn import Module
from torch import Tensor
import torch.nn as nn
from transformers import PretrainedConfig
from transformers.models.t5.modeling_t5 import T5Stack, T5ForConditionalGeneration

model_checkpoint = "t5-small"
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

ImportError: cannot import name 'Embedding' from 'transformers.models.t5.modeling_t5' (/home/john/anaconda3/envs/dl4nlp/lib/python3.9/site-packages/transformers/models/t5/modeling_t5.py)

In [3]:
def visualize_children(
    object,
    level : int = 0,
) -> None:
    """
    Prints the children of (object) and their children too, if there are any.
    Uses the current depth (level) to print things in a ordonnate manner.
    """
    print(f"{'   ' * level}{level}- {type(object).__name__}")
    try:
        for child in object.children():
            visualize_children(child, level + 1)
    except:
        pass

visualize_children(model)

0- T5ForConditionalGeneration
   1- Embedding
   1- T5Stack
      2- Embedding
      2- ModuleList
         3- T5Block
            4- ModuleList
               5- T5LayerSelfAttention
                  6- T5Attention
                     7- Linear
                     7- Linear
                     7- Linear
                     7- Linear
                     7- Embedding
                  6- T5LayerNorm
                  6- Dropout
               5- T5LayerFF
                  6- T5DenseActDense
                     7- Linear
                     7- Linear
                     7- Dropout
                     7- ReLU
                  6- T5LayerNorm
                  6- Dropout
         3- T5Block
            4- ModuleList
               5- T5LayerSelfAttention
                  6- T5Attention
                     7- Linear
                     7- Linear
                     7- Linear
                     7- Linear
                  6- T5LayerNorm
                  6- Dropout
         

In [52]:
def create_student_model(teacher: Module,
                         n: int):
    """Create a student model from a teacher model.

    Args:
        teacher: The teacher model.
        n: The fraction of the teacher model to keep.

    Returns:
        A student model.
    """
    config = teacher.config.to_dict()
    config['num_layers'] //= n
    config['num_decoder_layers'] //= n
    config = PretrainedConfig.from_dict(config)
    student_model = type(teacher)(config)
    student = student_model
    student.n = n
    init_student_weights(teacher, student)
    return student

def init_student_weights(teacher: Module,
                         student: Module):
    """Initialize the weights of a student model.

    Args:
        student: The student model.
        teacher: The teacher model.
    """
    student.shared.weight.data = teacher.shared.weight.data
    # Encoder
    for i in range(student.config.num_layers):
        student.encoder.block[i].load_state_dict(teacher.encoder.block[i * student.n].state_dict())
    # Decoder
    for i in range(student.config.num_decoder_layers):
        student.decoder.block[i].load_state_dict(teacher.decoder.block[i * student.n].state_dict())
    # Copy first embedding 